In [112]:
from cobra.flux_analysis import pfba
from os import getcwd
import math
import numpy as np
import pandas as pd
import holoviews as hv
from __future__ import unicode_literals
import matplotlib

In [113]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite, Model

In [114]:
matplotlib.interactive(True)

In [115]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [116]:
relative_directory = getcwd()
mcaps = read_sbml_model(relative_directory + '/../ModelFiles/xml/mcapsBath.xml')

## Reactions using Urea

In [117]:
mcaps.metabolites.urea_p.reactions

frozenset({<Reaction UREAabcpp at 0x119b7a090>,
           <Reaction UREAtex at 0x119b7a150>})

In [118]:
mcaps.reactions.DADK.gene_reaction_rule

'MCA2066'

In [119]:
for rxn in mcaps.metabolites.urea_c.reactions:
    print rxn.id, rxn.gene_reaction_rule

BLUB MCA2289
UREASE_GAPFILLING 
UREAabcpp MCA1662


In [120]:
mcaps.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_n2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

## Results Collection

In [121]:
df = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio"])

In [122]:
def df_record(solution, df, column):
    gr = solution.fluxes.BIOMASS_Mcapsulatus
    yx = gr/(abs((solution.fluxes.EX_ch4_e/1000)*16.04))
    data = [solution.fluxes.EX_ch4_e, solution.fluxes.EX_o2_e, solution.fluxes.EX_co2_e, solution.fluxes.EX_nh4_e,
            solution.fluxes.EX_n2_e, solution.fluxes.EX_no3_e, solution.fluxes.CYOR_q8ppi, gr, yx, 
            solution.fluxes.EX_o2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_nh4_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.EX_n2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_no3_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.ATPSipp/abs(solution.fluxes.EX_ch4_e)]
    df[column] = data
    return df

In [123]:
mcaps.reactions.EX_n2_e.bounds = 0, 1000
mcaps.reactions.ATPSNaipp.bounds = 0, 0
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000

In [124]:
def highlevel_stoichiometry(solution):
    boundary_rxns = set(mcaps.exchanges) & set(solution.reactions)
    for rxn in boundary_rxns:
        if rxn.flux < 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))
    for rxn in boundary_rxns:
        if rxn.flux > 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))

## Redox-Arm Only

Carbon-Limited, NO3, pMMO, Redox-Arm Only

In [125]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Redox-Arm Only')

In [126]:
highlevel_stoichiometry(solution)

(['no3_e'], -1.45857)
(['Co(2+)'], -1e-05)
(['Nickel'], -1e-05)
(['phosphate'], -0.09799)
(['dioxygen'], -27.8071)
(['sulfate'], -0.02999)
(['methane'], -18.46)
(['Ca(2+)'], -0.01244)
(['Cl(-)'], -0.03817)
(['Zinc'], -0.00018)
(['fe2_p'], -0.00109)
(['H(+)'], -1.45807)
(['K(+)'], -0.03142)
(['Na(+)'], -0.00697)
(['Mg(2+)'], -0.02198)
(['Manganese(2+)'], -0.0)
(['Cu2+'], -0.00027)
(['4-Hydroxy-benzyl alcohol'], 1e-05)
(['H2O'], 32.96016)
(['CO(2)'], 12.46802)
(['doxopa_c'], 0.00019)
(['hydrogen peroxide'], 0.05725)
(['carbon monoxide(1+)'], 1e-05)
(['alpha-ribazole'], 0.00019)


Carbon-Limited, NO3, sMMO, Redox-Arm Only

In [127]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO, Redox-Arm Only')

Carbon-Limited, NH3, pMMO, Redox-Arm Only

In [128]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Redox-Arm Only')

In [129]:
highlevel_stoichiometry(solution)

(['Co(2+)'], -2e-05)
(['Nickel'], -1e-05)
(['phosphate'], -0.1435)
(['dioxygen'], -27.86102)
(['sulfate'], -0.04393)
(['methane'], -18.46)
(['Ca(2+)'], -0.01818)
(['Cl(-)'], -0.0558)
(['Zinc'], -0.00026)
(['fe2_p'], -0.00159)
(['K(+)'], -0.04593)
(['Na(+)'], -0.01019)
(['Ammonium'], -2.13275)
(['Mg(2+)'], -0.03213)
(['Manganese(2+)'], -0.0)
(['Cu2+'], -0.00039)
(['4-Hydroxy-benzyl alcohol'], 1e-05)
(['H2O'], 33.26359)
(['CO(2)'], 9.69903)
(['doxopa_c'], 0.00028)
(['hydrogen peroxide'], 0.08361)
(['H(+)'], 2.13306)
(['carbon monoxide(1+)'], 1e-05)
(['alpha-ribazole'], 0.00028)


Carbon-Limited, NH3, sMMO, Redox-Arm Only

In [130]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO, Redox-Arm Only')

## Redox-Arm Parameter Exploration

In [131]:
mcaps.reactions.ATPSipp.metabolites

{<Metabolite adp_c at 0x119502550>: -1.0,
 <Metabolite atp_c at 0x119527490>: 1.0,
 <Metabolite h2o_c at 0x1195db3d0>: 1.0,
 <Metabolite h_c at 0x1195db790>: 3.0,
 <Metabolite h_p at 0x1195db890>: -4.0,
 <Metabolite pi_c at 0x1196675d0>: -1.0}

In [132]:
for r in np.arange(0,2,0.2):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        mcaps.reactions.ATPSipp.add_metabolites(
            {mcaps.metabolites.adp_c: -1.0, 
             mcaps.metabolites.atp_c: 1.0,
             mcaps.metabolites.h2o_c: 1.0,
             mcaps.metabolites.h_c: (3.0-r),
             mcaps.metabolites.h_p: -(4.0-r), 
             mcaps.metabolites.pi_c: -1.0}, combine=False)
        solution = pfba(mcaps)
#         print(mcaps.reactions.ATPSipp)
#         print(solution.fluxes.ATPSipp)

    df = df_record(solution, df, 'NO3, pMMO, Redox-Arm, H/ATP {}'.format(3-r))

## Direct Coupling Exploration

Carbon-Limited, NO3, pMMODC

In [133]:
for r in np.arange(0,6.5,0.5):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        mcaps.reactions.PMMODCipp.bounds = 0, 1000
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                r* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        solution = pfba(mcaps)
        print(solution.fluxes.MEOHDHipp,solution.fluxes.PMMODCipp, solution.fluxes.PMMOipp)

    df = df_record(solution, df, 'NO3, pMMODC, RedEff {}'.format(r))

(18.46, 18.46, 0.0)
(18.46, 12.306666666666668, 6.153333333333333)
(18.459999999999784, 9.229999999999892, 9.229999999999892)
(18.46, 7.384000000000001, 11.076000000000002)
(18.46, 6.153333333333334, 12.306666666666667)
(18.459999999999383, 5.274285714285536, 13.185714285713841)
(18.460000000008367, 4.615000000002092, 13.845000000006277)
(18.459999999999887, 4.102222222222197, 14.357777777777692)
(18.46000000000604, 3.6920000000012068, 14.768000000004827)
(18.459999999735018, 3.3563636363154576, 15.10363636341956)
(18.45999999999487, 3.0766666666658113, 15.383333333329057)
(18.45999999999818, 2.839999999999719, 15.619999999998456)
(18.460000000001152, 2.6371428571430218, 15.82285714285813)


## Uphill ET Parameter Exploration

Carbon-Limited, NO3, pMMO, Range (-5 - 0) 

In [134]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NO3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NH3, pMMO, Range (-5 - 0)

In [135]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NO3, pMMO, Uphill Unconstrained

In [136]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Uphill Unconstrained')

Carbon-Limited, NH4, pMMO, Uphill Unconstrained

In [137]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Uphill Unconstrained')

In [138]:
df

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.8","NO3, pMMO, Redox-Arm, H/ATP 1.6","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-1.846000e+01,-1.846000e+01,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.807105,-27.806966,-27.861020,-27.861020,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-22.265532,-22.869267,-23.250449,-2.416175e+01,-2.476929e+01,-25.203242,-25.528707,-25.781847,-25.984359,-26.150051,-26.288127,-26.404961,-26.505104,-2.533865e+01,-2.543738e+01,-2.553611e+01,-2.563485e+01,-25.733578,-25.832311,-25.931043,-26.029776,-26.128509,-26.227242,-26.325974,-26.424707,-26.523440,-26.622173,-26.720905,-26.819638,-26.918371,-27.017104,-27.115837,-27.214569,-27.313302,-27.412035,-27.510768,-27.609500,-27.708233,-25.407342,-25.505489,-25.603636,-25.701783,-25.799930,-25.898077,-25.996224,-26.094372,-26.192519,-26.290666,-26.388813,-26.486960,-26.585107,-26.683254,-26.781401,-26.879549,-26.977696,-27.

## 'Turning on' the uphill electron transfer

Using the value that was fit above

In [139]:
mcaps.reactions.CYOR_q8ppi.bounds = -2.8, 1000

Carbon-Limited, NO3, sMMO

In [140]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO')

Carbon-Limited, NO3, pMMO

In [141]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO')

Carbon-Limited, NH3, sMMO

In [142]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO')

Carbon-Limited, NH3, pMMO

In [143]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO')

Carbon-Limited, N2, sMMO

In [144]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, sMMO')

Carbon-Limited, N2, pMMO

In [145]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMO')

## Direct Coupling 

Carbon-Limited, NO3, pMMODC

In [146]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC')

Carbon-Limited, NH3, pMMODC

In [147]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH4, pMMODC')

Carbon-Limited, N2, pMMODC

In [148]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMODC')

## Effect of NH3 oxidation

Carbon-Limited, NH3, pMMO, NO2

In [149]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, 1/2*NO2')

Carbon-Limited, NH3, pMMO, NO2, Range(0-5)

In [150]:
np.arange(1,4.2,0.2)

array([1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4, 2.6, 2.8, 3. , 3.2, 3.4,
       3.6, 3.8, 4. ])

In [151]:
for r in np.arange(1,4.2,0.2):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
        mcaps.reactions.EX_nh4_e.flux_expression + r* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, 1/{}*NO2'.format(r))

Carbon-Limited, NO3, pMMO, NO2

In [152]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, 1/2*NO2')

O2-Limited, NH3, pMMO, 0.29

In [153]:
with mcaps:
    mcaps.objective = mcaps.reactions.EX_o2_e
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.BIOMASS_Mcapsulatus.bounds = 0.291, 0.291
    solution = pfba(mcaps)

df = df_record(solution, df, 'O2-Limited, NH3, pMMO, 0.29')

C-Limited, NO3, pMMO, KOTerminalOxidase

In [154]:
# mcaps.reactions.CYTAA3ipp

In [155]:
# with mcaps:
#     mcaps.reactions.EX_nh4_e.bounds = -1000, 0
#     mcaps.reactions.PMMOipp.bounds = 0, 1000
#     mcaps.reactions.GLNS.bounds = 0, 1000
#     mcaps.reactions.CYTAA3ipp.bounds = 0, 0
#     mcaps.reactions.CYTCBDipp.bounds = 0, 0
#     solution = pfba(mcaps)

# df = df_record(solution, df, 'NO3, pMMO, KOTerminalOxidase')

In [156]:
df.round(2)

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.8","NO3, pMMO, Redox-Arm, H/ATP 1.6","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained","NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO","NO3, pMMODC","NH4, pMMODC","N2, pMMODC","NH3, pMMO, 1/2*NO2","NH3, pMMO, 1/1.0*NO2","NH3, pMMO, 1/1.2*NO2","NH3, pMMO, 1/1.4*NO2","NH3, pMMO, 1/1.6*NO2","NH3, pMMO, 1/1.8*NO2","NH3, pMMO, 1/2.0*NO2","NH3, pMMO, 1/2.2*NO2","NH3, pMMO, 1/2.4*NO2","NH3, pMMO, 1/2.6*NO2","NH3, pMMO, 1/2.8*NO2","NH3, pMMO, 1/3.0*NO2","NH3, pMMO, 1/3.2*NO2","NH3, pMMO, 1/3.4*NO2","NH3, pMMO, 1/3.6*NO2","NH3, pMMO, 1/3.8*NO2","NH3, pMMO, 1/4.0*NO2","NO3, pMMO, 1/2*NO2","O2-Limited, NH3, pMMO, 0.29"
Methane UTR,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-2.58,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-17.84
Oxygen UTR,-27.81,-27.81,-27.86,-27.86,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-22.27,-22.87,-23.25,-24.16,-24.77,-25.20,-25.53,-25.78,-25.98,-26.15,-26.29,-26.40,-26.51,-25.34,-25.44,-25.54,-25.63,-25.73,-25.83,-25.93,-26.03,-26.13,-26.23,-26.33,-26.42,-26.52,-26.62,-26.72,-26.82,-26.92,-27.02,-27.12,-27.21,-27.31,-27.41,-27.51,-27.61,-27.71,-25.4

In [157]:
df.to_csv("../ComplementaryData/SupplementTable2.csv")

In [56]:
import matplotlib.pyplot as plt

In [57]:
plt.plot([1,2,3,4])

In [58]:
plt.draw()

In [59]:
plt.show()

In [60]:
import matplotlib.pyplot as plt

In [61]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['Growth Yield'].plot.bar(); plt.axhline(0, color='k')

In [62]:
df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']]

,"NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.025099,-26.424707,-27.861020,-26.486960,-28.782341,-26.772992
CO2 UTR,11.954014,11.559251,9.699031,8.370173,11.182966,9.386123
NH4 UTR,0.000000,0.000000,-2.132753,-2.456248,0.000000,0.000000
N2 UTR,0.000000,0.000000,0.000000,0.000000,-0.600096,-0.748272
NO3 UTR,-1.583804,-1.679905,0.000000,0.000000,0.000000,0.000000
Uphill Electron Transfer Flux,-1.583804,-2.800000,0.000000,-2.800000,1.409985,-2.800000
Growth Rate,0.193276,0.205004,0.260266,0.299743,0.081258,0.101322
Growth Yield,0.652744,0.692350,0.878985,1.012309,0.274428,0.342189
O2/CH4 Ratio,1.463982,1.431458,1.509264,1.434830,1.559173,1.450325


In [63]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7)); plt.axhline(0, color='k')

In [64]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, pMMODC','NH4, pMMODC','N2, pMMODC','NH3, pMMO, 1/2*NO2']].loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7))